# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
#iniciar a leitura do arquivo e aplicando metodologia para a remoção de linhas duplicadas.
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
# Verificando quantos valores 'missing' temos por cada variável.
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui
# Verificando quantos valores 'missing' temos por cada variável apenas para colunas de interesse para a análise.
sinasc[['LOCNASC','IDADEMAE','ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [4]:
# 4) seu código aqui
# Remoção de linhas não preenchidas.
sinasc.dropna(subset=['APGAR5'],inplace=True)

In [5]:
# Contagem de dados faltantes "missing" por variavel e as linhas.
sinasc[['LOCNASC','IDADEMAE','ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [6]:
# 5) seu código aqui
# Vamos assumir que, para as variáveis abaixo, os valores não preenchidos são iguais a 9 ( código 9 significa 'Ignorado').
sinasc.loc[sinasc['ESTCIVMAE'] == 'NA','ESTCIVMAE'] = 9
sinasc.loc[sinasc['CONSULTAS'] == 'NA','CONSULTAS'] = 9
sinasc

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,2604388.0,510340,1,32,2.0,12 anos ou mais,333115.0,1.0,0.0,...,5,2767632,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
27024,1,2752654.0,510675,1,19,1.0,8 a 11 anos,999992.0,0.0,0.0,...,5,2772678,ATIVO,MUNIC,Chupinguaia,Rondônia,-12.55684,-60.90185,340.0,5126.723
27025,1,9659366.0,510787,1,24,2.0,8 a 11 anos,421125.0,0.0,1.0,...,5,2776905,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
27026,1,9659366.0,510787,1,21,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2777105,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915


In [7]:
# 6) Seu código aqui
# Para a variável abaixo, os dados faltantes serão preenchidos pelo zero.
sinasc['QTDFILVIVO'].fillna(0)

0        0.0
1        1.0
2        2.0
3        0.0
4        1.0
        ... 
27023    1.0
27024    0.0
27025    0.0
27026    1.0
27027    1.0
Name: QTDFILVIVO, Length: 26925, dtype: float64

In [8]:
# 7) seu código aqui
# Substituição dos dados faltantes por "Não Informado", como são dados Qualitativos, não é assertivo tentar algum metódo de previsão,
# como uma conta de média ou mediana.
sinasc['ESCMAE'].fillna('Não informado')

0            8 a 11 anos
1            8 a 11 anos
2            8 a 11 anos
3        12 anos ou mais
4            8 a 11 anos
              ...       
27023    12 anos ou mais
27024        8 a 11 anos
27025        8 a 11 anos
27026        8 a 11 anos
27027        8 a 11 anos
Name: ESCMAE, Length: 26925, dtype: object

In [9]:
# Substituição dos dados faltantes por "Não Informado", como são dados Qualitativos, não é assertivo tentar algum metódo de previsão,
# como uma conta de média ou mediana.
sinasc['GESTACAO'].fillna('Não informado')

0        37 a 41 semanas
1        37 a 41 semanas
2        37 a 41 semanas
3        37 a 41 semanas
4        37 a 41 semanas
              ...       
27023    32 a 36 semanas
27024    37 a 41 semanas
27025    37 a 41 semanas
27026    32 a 36 semanas
27027    37 a 41 semanas
Name: GESTACAO, Length: 26925, dtype: object

In [10]:
# Substituição dos dados faltantes por "Não Informado", como são dados Qualitativos, não é assertivo tentar algum metódo de previsão,
# como uma conta de média ou mediana.
sinasc['GRAVIDEZ'].fillna('Não informado')

0        Única
1        Única
2        Única
3        Única
4        Única
         ...  
27023    Única
27024    Única
27025    Única
27026    Única
27027    Única
Name: GRAVIDEZ, Length: 26925, dtype: object

In [11]:
# 8) seu código aqui
# Categorização e Sub Categoricação dos dados APGAR.
sinasc.loc[sinasc['APGAR5']<= 3, 'Apgar_Categoria'] = 'Asfixia_Severa'
sinasc.loc[(sinasc['APGAR5']>= 4) & (sinasc['APGAR5']<= 5), 'Apgar_Categoria'] = 'Asfixia_Moderada'
sinasc.loc[(sinasc['APGAR5']>= 6) & (sinasc['APGAR5']<= 7), 'Apgar_Categoria'] = 'Asfixia_Leve'
sinasc.loc[sinasc['APGAR5']>= 8, 'Apgar_Categoria'] = 'Normal'

In [12]:
sinasc.loc[sinasc['APGAR5']>= 8, 'Apgar_Categoria_2'] = 'Normal'
sinasc.loc[sinasc['APGAR5']<= 7, 'Apgar_Categoria_2'] = 'Asfixia'

In [17]:
# Frequencia de Apgar 5
sinasc[['Apgar_Categoria','Apgar_Categoria_2']]

,Apgar_Categoria,Apgar_Categoria_2
0,Normal,Normal
1,Normal,Normal
2,Normal,Normal
3,Normal,Normal
4,Normal,Normal
...,...,...
27023,Normal,Normal
27024,Normal,Normal
27025,Normal,Normal
27026,Normal,Normal


In [13]:
sinasc

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea,Apgar_Categoria,Apgar_Categoria_2
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,Normal,Normal
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,Normal,Normal
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,Normal,Normal
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273,Normal,Normal
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,2604388.0,510340,1,32,2.0,12 anos ou mais,333115.0,1.0,0.0,...,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,Normal,Normal
27024,1,2752654.0,510675,1,19,1.0,8 a 11 anos,999992.0,0.0,0.0,...,ATIVO,MUNIC,Chupinguaia,Rondônia,-12.55684,-60.90185,340.0,5126.723,Normal,Normal
27025,1,9659366.0,510787,1,24,2.0,8 a 11 anos,421125.0,0.0,1.0,...,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,Normal,Normal
27026,1,9659366.0,510787,1,21,2.0,8 a 11 anos,999992.0,1.0,0.0,...,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,Normal,Normal


In [14]:
# 9) seu código aqui
# Reunindo todos os titulos das colunas para a padronização dos textos.
print(sinasc.columns)

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea',
       'Apgar_Categoria

In [15]:
# Renomear os titulos das colunas, para que fique conforme o padrão snake case e underscore.
# Dicionário retirado da fonte de Dados do SUS : https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacao-sobre-nascidos-vivos/dicionario-de-variaveis/
sinasc.rename(columns={'ORIGEM':'Origem', 'CODESTAB':'Codigo_Estabelecimento','CODMUNNASC': 'Codigo_Municipio_Nascimento','LOCNASC':'Local_Nascimento_Mae','IDADEMAE':'Idade_Mae', 'ESTCIVMAE':'Estado_Civil_Mae','ESCMAE':'Escolaridade_Mae','CODOCUPMAE':'Codigo_Ocupacao_Mae','QTDFILVIVO':'Quantidade_Filhos_Vivos','QTDFILMORT':'Quantidade_Filhos_Mortos','CODMUNRES':'Codigo_Municipio_Residencia_Mae','GESTACAO':'Gestacao','GRAVIDEZ':'Gravidez','PARTO':'Parto','CONSULTAS':'Consultas','DTNASC':'Data_Nascimento','HORANASC':'Hora_Nascimento','SEXO':'Sexo', 'APGAR1':'Apgar_1', 'APGAR5':'Apgar_5', 'RACACOR':'Raca_Cor', 'PESO':'Peso','IDANOMAL':'Anomalia_Congenita','DTCADASTRO':'Data_Cadastro','CODANOMAL':'Codigo_Anomalia','NUMEROLOTE':'Numero_Lote', 'VERSAOSIST':'Versao_Sistema', 'DTRECEBIM':'Data_Recebimento', 'DIFDATA':'Diferença_Data','DTRECORIGA':'Data_Recebimento_Lote', 'NATURALMAE':'Natural_Mae', 'CODMUNNATU':'Codigo_Municipio_Natural', 'CODUFNATU':'Codigo_UF_Natural', 'ESCMAE2010':'Escolaridade_Mae_2010','SERIESCMAE':'Serie_Escolar_Mae','DTNASCMAE':'Data_Nascimento_Mae','RACACORMAE':'Raca_Cor_Mae','QTDGESTANT':'Quantidade_Gestacoes','QTDPARTNOR':'Quantidade_Parto_Normal','QTDPARTCES':'Quantidade_Parto_Cesarea','IDADEPAI':'Idade_Pai','DTULTMENST':'Data_Ultima_Menstruacao','SEMAGESTAC':'Semana_Gestacao','TPMETESTIM':'TP_Metodo_Estimado','CONSPRENAT':'Consultas_Pre_Natal', 'MESPRENAT':'Mes_Pre_Natal', 'TPAPRESENT':'Tipo_Apresentacao', 'STTRABPART':'Trabalho_Parto_Induzido', 'STCESPARTO':'Trabalho_Parto_Anterior_Cesarea','TPNASCASSI':'Nascimento_Assistido', 'TPFUNCRESP':'Funcao_Funcionario_Preenchimento', 'TPDOCRESP':'Tipo_Documento_Responsavel', 'DTDECLARAC':'Data_Declaracao', 'ESCMAEAGR1':'Escolaridade_Mae_Agregada','STDNEPIDEM':'Status_DO_Epidemiologica','STDNNOVA':'Status_DO_Nova','CODPAISRES':'Codigo_Pais_Residente','TPROBSON':'Codigo_Grupo_Robson','PARIDADE':'Paridade','KOTELCHUCK':'Kotelchuck','CONTADOR':'Contador','munResStatus':'Municipio_Residente_Status','munResTipo':'Municipio_Residente_Tipo','munResNome':'Municipio_Residente_Nome','munResUf':'Municipio_Residente_UF','munResLat':'Municipio_Residente_Latitude','munResLon':'Municipio_Residente_Longitude','munResAlt':'Municipio_Residente_Altitude','munResArea':'Municipio_Residente_Area'}, inplace = True)
print(sinasc.columns)

Index(['Origem', 'Codigo_Estabelecimento', 'Codigo_Municipio_Nascimento',
       'Local_Nascimento_Mae', 'Idade_Mae', 'Estado_Civil_Mae',
       'Escolaridade_Mae', 'Codigo_Ocupacao_Mae', 'Quantidade_Filhos_Vivos',
       'Quantidade_Filhos_Mortos', 'Codigo_Municipio_Residencia_Mae',
       'Gestacao', 'Gravidez', 'Parto', 'Consultas', 'Data_Nascimento',
       'Hora_Nascimento', 'Sexo', 'Apgar_1', 'Apgar_5', 'Raca_Cor', 'Peso',
       'Anomalia_Congenita', 'Data_Cadastro', 'Codigo_Anomalia', 'Numero_Lote',
       'Versao_Sistema', 'Data_Recebimento', 'Diferença_Data',
       'Data_Recebimento_Lote', 'Natural_Mae', 'Codigo_Municipio_Natural',
       'Codigo_UF_Natural', 'Escolaridade_Mae_2010', 'Serie_Escolar_Mae',
       'Data_Nascimento_Mae', 'Raca_Cor_Mae', 'Quantidade_Gestacoes',
       'Quantidade_Parto_Normal', 'Quantidade_Parto_Cesarea', 'Idade_Pai',
       'Data_Ultima_Menstruacao', 'Semana_Gestacao', 'TP_Metodo_Estimado',
       'Consultas_Pre_Natal', 'Mes_Pre_Natal', 'Tipo_Ap